In [1]:
import numpy as np
import tensorflow as tf
import reader_ptb

In [2]:
DATA_PATH = "../../datasets/PTB_data/" #存数据路径
HIDDEN_SIZE = 200
NUM_LAYERS = 2
VOCAB_SIZE = 10000

LEARNING_RATE = 1.0
TRAIN_BATCH_SIZE  = 20
TRAIN_NUM = STEP = 35

EVAL_BATCH_SIZE = 1
EVAL_NUM_STEP = 1
NUM_EPOCH = 2
KEEP_PROB = 0.5
MAX_GRAD_NORM =5


In [4]:
class PTBModel:
    def __init__(self,is_training,batch_size,num_steps):
        self.batch_size = batch_size 
        self.num_steps = num_steps
        
        self.input_data = tf.placeholder(tf.int32,[batch_size,num_steps])
        self.targets = tf.placeholder(tf.int32,[batch_size,num_steps])
        cell = tf.contrib.rnn.MultiRNNCell([self.lstm_cell(hidden_size,keep_prob,is_training) for _ in range(0,NUM_LAYERS)],state_is_tuple=True)
        
        self.initial_state = cell.zero_state([batch_size,num_steps],tf.float32)
        embedding = tf.get_variable("embedding",[VOCAB_SIZE,HIDDEN_SIZE])
        # 将原本单词ID转为单词向量。
        inputs = tf.nn.embedding_lookup(embedding,self.input_data)#size(batch_size,HIDDEN_SIZE),
        
        if is_training:
            inputs = tf.nn.dropout(inputs,KEEP_PROB)
        
        outputs = []
        state = self.initial_state
        with tf.variable_scope("RNN"):
            for time_step in range(num_steps):
                if time_step > 0 :tf.get_variable_scope().resue_variables()
                cell_output,state = cell(inputs[:,time_step,:],state)
                outputs.append(cell_output)
        #concat(1) 是按照第二维度进行拼接，tf.reshape(,[-1,])将每一时刻的输出在第1维上拼接（上图），这样每一行就完整的表示了一个序列。reshape后的结构如下图：
        output = tf.reshape(tf.concat(outputs,-1),[-1,HIDDEN_SIZE])
        weight = tf.get_variable("weight",[HIDDEN_SIZE,VOCAB_SIZE])
        bias = tf.get_variable("bias",[VOCAB_SIZE])
        
        logits = tf.matmul(output,weight)+bias
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            [logits],
            [tf.shape(self.targets,[-1])],
            [tf.ones([batch_size*num_steps],dtype=tf.float32)])
        self.cost = tf.reduce_sum(loss)/batch_size#计算平均值
        self.final_state = state #最终状态
        
        if not is_training :return 
        train_variables = tf.trainable_variables()        
        grads,_ = tf.clip_by_global_norm(tf.gradients(self.cost,train_variables),MAX_GRAD_NORM)
        optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE)
        self.train_op = optimizer.apply_gradients(zip(grads, trainable_variables))  
    def lstm_cell(self,hidden_size,keep_prob,is_training):
        cell = tf.contrib.rnn.BasicLSTMCell(hidden_size,resue=tf.get_variable_scope().resue())
        if is_training:
            return tf.contrib.rnn.DropoutWrapper(cell,output_keep_prob=keep_prob)
        return cell

def run_epoch(session,model,data,train_op,output_log,epoch_size):
    total_cost = 0.0
    iters = 0
    state = session.run(model.initial_state)
    #训练一个epoch
    for step in range(epoch_size):
        x,y = session.run(data)
        cost,state ,_ = session.run([model_cost,model.final_state,train_op],{model.input_data:x,model.targets:y,model.initial_state:state})
        total_cost += cost
        iters += model.num_steps
        
        if output_log and step %100 ==0:
            print("After %d steps, perplexity is %.3f" % (step, np.exp(total_costs / iters)))
    return np.exp(total_costs / iters)

def main():
    train_data,valid_data,test_data ,_ = reader_ptb.ptb_raw_data(DATA_PATH)
    
    # 计算一个epoch需要训练的次数
    train_data_len = len(train_data)
    train_batch_len = train_data_len // TRAIN_BATCH_SIZE
    train_epoch_size = (train_data_len-1)//TRAIN_NUM_STEP
    
    valid_data_len = len(valid_data)
    valid_batch_len = valid_batch_len // EVAL_BATCH_SIZE
    valid_epoch_size = (valid_batch_len - 1) // EVAL_NUM_STEP
    
    test_data_len = len(test_data)
    test_batch_len = test_data_len // EVAL_BATCH_SIZE
    test_epoch_size = (test_batch_len - 1) // EVAL_NUM_STEP
    
    
    initalizer = tf.random_normal_initializer(-0.05,0.05)
    with tf.variable_scope("language_model",reuse=None,initializer=initializer):
        train_model = PTBModel(True,TRAIN_BATCH_SIZE,TRAIN_NUM_STEP)
    with tf.variable_scope("language_model", reuse=True, initializer=initializer):
        eval_model = PTBModel(False, EVAL_BATCH_SIZE, EVAL_NUM_STEP)
    
    with tf.Session() as sess:
        tf.global_variables_initializer()
        
        train_queue = reader_ptb.ptb_producer(train_data, train_model.batch_size, train_model.num_steps)
        eval_queue = reader_ptb.ptb_producer(valid_data, eval_model.batch_size, eval_model.num_steps)
        test_queue = reader_ptb.ptb_producer(test_data, eval_model.batch_size, eval_model.num_steps)
        
        coord = tf.train.Coordinator()    
        threads = tf.train.start_queue_runners(sess=session, coord=coord)        
        for i in range(NUM_EPOCH):
            print("In iteration: %d" % (i + 1))
            run_epoch(session, train_model, train_queue, train_model.train_op, True, train_epoch_size)

            valid_perplexity = run_epoch(session, eval_model, eval_queue, tf.no_op(), False, valid_epoch_size)
            print("Epoch: %d Validation Perplexity: %.3f" % (i + 1, valid_perplexity))

        test_perplexity = run_epoch(session, eval_model, test_queue, tf.no_op(), False, test_epoch_size)
        print("Test Perplexity: %.3f" % test_perplexity)
        #saver.save(session, 'model/lm')
        coord.request_stop()
        coord.join(threads)
    